### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
access_token = os.getenv('ACCESS_TOKEN')

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f'{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}'
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [4]:
#get response
response = requests.get(query_url)
response

#retrieve reviews as json 
reviews = response.json()

#print results to start exploring data
print(json.dumps(reviews, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url_page = f'{query_url}&page={page}'

    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url_page).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews['response']['docs']:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f'Page {page} retrieved.')
    except:
        # Print the page number that had no results then break from the loop
        print(f'Page {page} had no results.')
    

Page 0 retrieved.
Page 1 retrieved.
Page 2 retrieved.
Page 3 retrieved.
Page 4 retrieved.
Page 5 retrieved.
Page 6 retrieved.
Page 7 retrieved.
Page 8 retrieved.
Page 9 retrieved.
Page 10 retrieved.
Page 11 retrieved.
Page 12 retrieved.
Page 13 retrieved.
Page 14 retrieved.
Page 15 retrieved.
Page 16 retrieved.
Page 17 retrieved.
Page 18 retrieved.
Page 19 retrieved.


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.DataFrame(pd.json_normalize(reviews_list))
reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
#reviews_df['title'] = reviews_df['headline.print_headline']

In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [10]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].to_list()
titles


['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [12]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1 

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter%50 == 0:
        time.sleep(1)
        print('The application is sleeping')
    
    # Add 1 to the request counter
    request_counter += 1
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        # Perform a "GET" request for The Movie Database
        movie_title = requests.get(url+title+tmdb_key_string).json()
        movie_id = movie_title['results'][0]['id']


        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}"
        headers = {
         "accept": "application/json",
        "Authorization": f"Bearer {access_token}"
        }

        # Make a request for a the full movie details
        movie_results = requests.get(movie_url, headers=headers).json()
        
        # Extract the genre names into a list
        genres = []
        for genre in movie_results['genres']:
            genres.append(
                genre['name']
            )
        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in movie_results['spoken_languages']:
            spoken_languages.append(
                language['english_name']
            )
        # Extract the production_countries' name into a list
        production_countries = []
        for country in movie_results['production_countries']:
            production_countries.append(
                country['name']
            )

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            'title': movie_results['title'],
            'original_title': movie_results['original_title'],
            'budget': movie_results['budget'],
            'original_language': movie_results['original_language'],
            'homepage': movie_results['homepage'],
            'overview': movie_results['overview'],
            'popularity': movie_results['popularity'],
            'runtime': movie_results['runtime'],
            'revenue': movie_results['revenue'],
            'release_date': movie_results['release_date'],
            'vote_average': movie_results['vote_average'],
            'vote_count': movie_results['vote_count'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
            })

        
        # # Print out the title that was found
        # movie_title = movie_result['original_title']
        print(f'Movie {title} was found!')
    except:
        print(f'Movie {title} not found.')


Movie The Attachment Diaries was found!
Movie What’s Love Got to Do With It?’ Probably a Lo not found.
Movie You Can Live Forever was found!
Movie A Tourist’s Guide to Love was found!
Movie Other People’s Children was found!
Movie One True Loves was found!
Movie The Lost Weekend: A Love Story was found!
Movie A Thousand and One was found!
Movie Your Place or Mine was found!
Movie Love in the Time of Fentanyl was found!
Movie Pamela, a Love Story was found!
Movie In From the Side was found!
Movie After Love was found!
Movie Alcarràs was found!
Movie Nelly & Nadine was found!
Movie Lady Chatterley’s Lover was found!
Movie The Sound of Christmas was found!
Movie The Inspection was found!
Movie Bones and All was found!
Movie My Policeman was found!
Movie About Fate was found!
Movie Waiting for Bojangles was found!
Movie I Love My Dad was found!
Movie A Love Song was found!
Movie Alone Together was found!
Movie Art of Love was found!
Movie The Wheel was found!
Movie Thor: Love and Thunder w

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 5.018,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [14]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head(5)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",5.018,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",32.061,96,15055,2023-03-24,6.700,33,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",17.444,96,0,2023-04-21,6.300,158,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",5.933,104,84178,2022-09-21,6.842,184,"[Drama, Comedy]","[English, French]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,13.177,100,37820,2023-04-07,6.406,69,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"


### Merge and Clean the Data for Export

In [20]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df,reviews_df, on='title')
merged_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",5.018,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",32.061,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,13.177,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,4.466,95,77145,2023-04-13,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.883,116,3395595,2023-03-31,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [21]:
# Drop "byline.person" column
merged_df.drop('byline.person',axis=1, inplace=True)
merged_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",5.018,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",32.061,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,13.177,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,4.466,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.883,116,3395595,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [18]:
# Delete duplicate rows and reset index
clean_merged_df = merged_df.drop_duplicates().reset_index(drop=True)
clean_merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",5.018,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",32.061,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,13.177,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,4.466,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.883,116,3395595,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,In Search of Fellini,In Search of Fellini,0,en,,"Lucy, a small-town girl from Ohio, discovers t...",4.288,93,0,2017-09-15,...,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,None
120,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",18.187,130,0,2017-01-11,...,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
121,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,7.325,83,0,2017-01-14,...,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
122,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.855,103,0,2016-12-02,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None


In [22]:
# Export data to CSV without the index
clean_merged_df.to_csv('output/collected_data.csv',index=False)